## Предобработка данных

In [1]:
# Подключение к Google drive

# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
import numpy as np
import pandas as pd

In [3]:
# Загрузим набор данных

# df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/freMPL-R.csv', low_memory=False)
df = pd.read_csv('freMPL-R.csv', low_memory=False)
df = df.loc[df.Dataset.isin([5, 6, 7, 8, 9])]
df.drop('Dataset', axis=1, inplace=True)
df.dropna(axis=1, how='all', inplace=True)
df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115155 entries, 0 to 115154
Data columns (total 20 columns):
Exposure             115155 non-null float64
LicAge               115155 non-null int64
RecordBeg            115155 non-null object
RecordEnd            59455 non-null object
Gender               115155 non-null object
MariStat             115155 non-null object
SocioCateg           115155 non-null object
VehUsage             115155 non-null object
DrivAge              115155 non-null int64
HasKmLimit           115155 non-null int64
BonusMalus           115155 non-null int64
ClaimAmount          115155 non-null float64
ClaimInd             115155 non-null int64
ClaimNbResp          115155 non-null float64
ClaimNbNonResp       115155 non-null float64
ClaimNbParking       115155 non-null float64
ClaimNbFireTheft     115155 non-null float64
ClaimNbWindscreen    115155 non-null float64
OutUseNb             115155 non-null float64
RiskArea             115155 non-null float64
dtypes

В предыдущем уроке мы заметили отрицательную величину убытка для некоторых наблюдений. Заметим, что для всех таких полисов переменная "ClaimInd" принимает только значение 0. Поэтому заменим все соответствующие значения "ClaimAmount" нулями.

In [4]:
NegClaimAmount = df.loc[df.ClaimAmount < 0, ['ClaimAmount','ClaimInd']]
print('Unique values of ClaimInd:', NegClaimAmount.ClaimInd.unique())
NegClaimAmount.head()

Unique values of ClaimInd: [0]


,ClaimAmount,ClaimInd
82,-74.206042,0
175,-1222.585196,0
177,-316.288822,0
363,-666.758610,0
375,-1201.600604,0


In [5]:
df.loc[df.ClaimAmount < 0, 'ClaimAmount'] = 0

Перекодируем переменные типа `object` с помощью числовых значений

In [6]:
def SeriesFactorizer(series):
    series, unique = pd.factorize(series)
    reference = {x: i for x, i in enumerate(unique)}
    print(reference)
    return series, reference

In [7]:
df.Gender, GenderRef = SeriesFactorizer(df.Gender)

{0: 'Male', 1: 'Female'}


In [8]:
df.MariStat, MariStatRef = SeriesFactorizer(df.MariStat)

{0: 'Other', 1: 'Alone'}


Для переменных, содержащих более 2 значений, различия между которыми не могут упорядочены, используем фиктивные переменные (one-hot encoding).

**NB**: В H2O не рекомендуется использовать one-hot encoding, поскольку данный фреймворк корректно работает с категориальными признаками, тогда как применение one-hot encoding приводит к неэффективности. Тем не менее, используем здесь фиктивные переменные, чтобы в дальнейшем сохранить возможность сравнения результатов построенных моделей.

In [9]:
list(df.VehUsage.unique())

['Professional', 'Private+trip to office', 'Private', 'Professional run']

In [10]:
VU_dummies = pd.get_dummies(df.VehUsage, prefix='VehUsg', drop_first=False)
VU_dummies.head()

,VehUsg_Private,VehUsg_Private+trip to office,VehUsg_Professional,VehUsg_Professional run
0,0,0,1,0
1,0,0,1,0
2,0,1,0,0
3,0,1,0,0
4,1,0,0,0


In [11]:
df = pd.concat([df, VU_dummies], axis=1)

Фактор "SocioCateg" содержит информацию о социальной категории в виде кодов классификации CSP. Агрегируем имеющиеся коды до 1 знака, а затем закодируем их с помощью one-hot encoding.

[Wiki](https://fr.wikipedia.org/wiki/Professions_et_cat%C3%A9gories_socioprofessionnelles_en_France#Cr%C3%A9ation_de_la_nomenclature_des_PCS)

[Более подробный классификатор](https://www.ast74.fr/upload/administratif/liste-des-codes-csp-copie.pdf)

In [12]:
df['SocioCateg'].unique()

array(['CSP50', 'CSP55', 'CSP60', 'CSP48', 'CSP6', 'CSP66', 'CSP1',
       'CSP46', 'CSP21', 'CSP47', 'CSP42', 'CSP37', 'CSP22', 'CSP3',
       'CSP49', 'CSP20', 'CSP2', 'CSP40', 'CSP7', 'CSP26', 'CSP65',
       'CSP41', 'CSP17', 'CSP57', 'CSP56', 'CSP38', 'CSP51', 'CSP59',
       'CSP30', 'CSP44', 'CSP61', 'CSP63', 'CSP45', 'CSP16', 'CSP43',
       'CSP39', 'CSP5', 'CSP32', 'CSP35', 'CSP73', 'CSP62', 'CSP52',
       'CSP27', 'CSP24', 'CSP19', 'CSP70'], dtype=object)

In [13]:
df['SocioCateg'] = df.SocioCateg.str.slice(0,4)

In [14]:
SocCatSize = pd.DataFrame(df.groupby('SocioCateg').size().sort_values(), columns=['Frequency'])
SocCatSize

,Frequency
SocioCateg,
CSP7,14
CSP3,1210
CSP1,2740
CSP2,3254
CSP4,7648
CSP6,24833
CSP5,75456


In [15]:
df = pd.concat([df, pd.get_dummies(df.SocioCateg)], axis=1)

Теперь, когда большинство переменных типа `object` обработаны, исключим их из набора данных за ненадобностью.

In [16]:
df = df.select_dtypes(exclude=['object'])

Также создадим такую переменную, как квадрат возраста.

In [17]:
df['DrivAgeSq'] = df.DrivAge.apply(lambda x: x**2)
df.head()

,Exposure,LicAge,Gender,MariStat,DrivAge,HasKmLimit,BonusMalus,ClaimAmount,ClaimInd,ClaimNbResp,...,VehUsg_Professional,VehUsg_Professional run,CSP1,CSP2,CSP3,CSP4,CSP5,CSP6,CSP7,DrivAgeSq
0,0.083,332,0,0,46,0,50,0.0,0,0.0,...,1,0,0,0,0,0,1,0,0,2116
1,0.916,333,0,0,46,0,50,0.0,0,0.0,...,1,0,0,0,0,0,1,0,0,2116
2,0.550,173,0,0,32,0,68,0.0,0,0.0,...,0,0,0,0,0,0,1,0,0,1024
3,0.089,364,1,0,52,0,50,0.0,0,0.0,...,0,0,0,0,0,0,1,0,0,2704
4,0.233,426,0,0,57,0,50,0.0,0,0.0,...,0,0,0,0,0,0,0,1,0,3249


Для моделирования частоты убытков сгенерируем показатель как сумму индикатора того, что убыток произошел ("ClaimInd") и количества заявленных убытков по различным видам ущерба за 4 предшествующих года ("ClaimNbResp", "ClaimNbNonResp", "ClaimNbParking", "ClaimNbFireTheft", "ClaimNbWindscreen").

В случаях, если соответствующая величина убытка равняется нулю, сгенерированную частоту также обнулим.

In [18]:
df['ClaimsCount'] = df.ClaimInd + df.ClaimNbResp + df.ClaimNbNonResp + df.ClaimNbParking + df.ClaimNbFireTheft + df.ClaimNbWindscreen
df.loc[df.ClaimAmount == 0, 'ClaimsCount'] = 0
df.drop(["ClaimNbResp", "ClaimNbNonResp", "ClaimNbParking", "ClaimNbFireTheft", "ClaimNbWindscreen"], axis=1, inplace=True)

In [19]:
ClaimsCountGroup = pd.DataFrame(df.groupby('ClaimsCount').size(), columns=['Policies'])
ClaimsCountGroup

,Policies
ClaimsCount,
0.0,104286
1.0,3339
2.0,3529
3.0,2310
4.0,1101
5.0,428
6.0,127
7.0,26
8.0,6


In [20]:
# import plotly.express as px
# fig = px.scatter(df, x='ClaimsCount', y='ClaimAmount', title='Зависимость между частотой и величиной убытков')
# fig.show()

Для моделирования среднего убытка можем рассчитать его как отношение величины убытков к их частоте.

In [21]:
dfAC = df[df.ClaimsCount > 0].copy()
dfAC['AvgClaim'] = dfAC.ClaimAmount/dfAC.ClaimsCount

## Разделение набора данных на обучающую, валидационную и тестовую выборки

In [22]:
from sklearn.model_selection import train_test_split

In [23]:
# Разбиение датасета для частоты на train/val/test

x_train_c, x_test_c, y_train_c, y_test_c = train_test_split(df.drop(['ClaimInd', 'ClaimAmount', 'ClaimsCount'], axis=1), df.ClaimsCount, test_size=0.15, random_state=1)
x_train_c, x_valid_c, y_train_c, y_valid_c = train_test_split(x_train_c, y_train_c, test_size=0.15, random_state=1)

In [24]:
# Разбиение датасета для среднего убытка на train/val/test 

x_train_ac, x_test_ac, y_train_ac, y_test_ac = train_test_split(dfAC.drop(['ClaimInd', 'ClaimAmount', 'ClaimsCount', 'AvgClaim'], axis=1), dfAC.AvgClaim, test_size=0.15, random_state=1)
x_train_ac, x_valid_ac, y_train_ac, y_valid_ac = train_test_split(x_train_ac, y_train_ac, test_size=0.15, random_state=1)

## Обобщенные линейные модели (Generalized Linear Models, GLM)

### Теория

Пусть $y$ – целевая переменная, $X$ – матрица объясняющих переменных, $\beta$ – вектор параметров модели.

Матрица $X$ составлена из всех векторов наблюдений $x_i$, каждый из которых представляет собой объясняющую переменную.

Основные компоненты обобщенной линейной модели:
* Систематическая компонента $\eta$:
    * $\eta = X\beta \hspace{10pt}(=\beta_0+\beta_1x_1+\beta_2x_2+\dots+\beta_nx_n)$.
* Случайная компонента $y$:
    * Элементы вектора $y$ – независимые одинаково распределенные случайные величины, имеющие функцию плотности распределения $f(y;\theta,\phi)$ из экспоненциального семейства.
    * Распределения из экспоненциального семейства имеют параметры $\theta$ (характеристика среднего) и $\phi$ (характеристика дисперсии). В общем виде данные распределения могут быть определены:
    $$f_i(y_i;\theta_i,\phi)=\exp\left\lbrace \frac{y_i\theta_i-b(\theta_i)}{a_i(\phi)} + c(y_i, \phi) \right\rbrace,$$
    где $a_i(\phi)$, $b(\theta_i)$ и $c(y_i, \phi)$ некоторые функции.
    * Для распределений из данного семейства дисперсия является функцией от среднего.
    * Экспоненциальное семейство включает распределения нормальное, экспоненциальное, Пуассона, гамма, хи-квадрат, бета и другие.
* Функция связи $g$:
    * $\mathbb{E}\left[y\right]=\mu=g^{-1}\left(\eta\right)$, $\mu$ – математическое ожидание $y$;
    * $g$ – монотонная дифференцируемая функция.




#### GLM с распределением Пуассона

Регрессия Пуассона обычно используется в случаях, когда зависимая переменная представляет собой счетные значения и ошибки предполагаются распределенными в соответствии с распределением Пуассона. Зависимая переменная должна быть неотрицательной.

Функции связи между таргетом и объясняющими переменными предполагается логарифмической:
$$g(\eta) = \ln(\eta) \Rightarrow \hat {y} = e^{x{^T}\beta + {\beta_{0}}}.$$

Модель оценивается методом максимального правдоподобия, функция логарифма правдоподобия с учетом штрафа регуляризации эластичной сети имеет вид:
$$\max_{\beta,\beta_0} \frac{1}{N} \sum_{i=1}^{N} \Big( y_i(x_{i}^{T}\beta + \beta_0) - e^{x{^T_i}\beta + {\beta_0}} \Big)- \lambda \Big( \alpha||\beta||_1 + \dfrac {1} {2}(1 - \alpha)||\beta||^2_2 \Big),$$

где 
* $\lambda$ – параметр, отвечающий за силу регуляризации. $\lambda\in\mathbb{R}^{+}$;
* $\alpha$ – параметр, отвечающий за распределение штрафов регуляризации между нормой 1 ($\ell_1$) и нормой 2 ($\ell_2$). $\alpha\in[0,1]$;
* $||\beta||{_1}$ – штраф регуляризации $\ell_1$ (LASSO). $||\beta||{_1} = \sum{^p_{k=1}} |\beta{_k}|$;
* $||\beta||{_2}$ – штраф регуляризации $\ell_2$ (Ridge). $||\beta||{_2} = \sum{^p_{k=1}} \beta{^2_k}$.

Тогда соответствующая метрика _Deviance_ имеет вид:
$$D = -2 \sum_{i=1}^{N} \big( y_i \text{ln}(y_i / \hat {y}_i) - (y_i - \hat {y}_i) \big).$$

##### Вывод функции правдоподобия для GLM с распределением Пуассона (без регуляризации)

**NB:** Ниже $\lambda$ не имеет отношения к вышеупомянутому одноименному параметру регуляризации.

Напомним, что функция вероятности для распределения Пуассона имеет вид:
$$p(k;\lambda) = \frac{\lambda^k e^{-\lambda}}{k!}, \hspace{10pt} \lambda\in\mathbb{R}^{+}.$$
Также, для распределения Пуассона справедливо, что:
$$\mathbb{E}\left[k\right] = Var(k) = \lambda.$$
Тогда, для оценивания коэффициентов нашей модели необходимо максимизировать правдоподобие (совместную условную вероятность при имеющихся данных), что данные имеют распределение Пуассона:
$$p(y_1,\dots,y_n|x_1,\dots,x_n;\beta_0,\beta) = \prod_{i=1}^{N}\frac{e^{y_i(x_i{^T}\beta + {\beta_{0}})} e^{-e^{x_i{^T}\beta + {\beta_{0}}}}}{y_i!} = L(\beta_0,\beta|y_1,\dots,y_n,x_1,\dots,x_n).$$
Для упрощения задачи оптимизации перейдем к логарифму правдоподобия:
$$\ell(\beta_0,\beta|y_1,\dots,y_n,x_1,\dots,x_n) = \sum_{i=1}^{N}\left(y_i(x_i{^T}\beta + {\beta_{0}}) -e^{x_i{^T}\beta + {\beta_{0}}}-\ln(y_i!)\right).$$
Поскольку величина $\ln(y_i!)$ не зависит от выбора параметров, можно упростить задачу:
$$\ell(\beta_0,\beta|y_1,\dots,y_n,x_1,\dots,x_n) = \sum_{i=1}^{N}\left(y_i(x_i{^T}\beta + {\beta_{0}}) -e^{x_i{^T}\beta + {\beta_{0}}}\right).$$
Далее численно решается задача оптимизации для определения параметров модели:
$$\frac{\partial \ell(\beta_0,\beta|y_1,\dots,y_n,x_1,\dots,x_n)}{\partial \beta_0} = 0,\\\frac{\partial \ell(\beta_0,\beta|y_1,\dots,y_n,x_1,\dots,x_n)}{\partial \beta} = 0.$$
Обычно минимизируется отрицательное правдоподобие, которое является выпуклой функцией.

##### Вывод метрики Deviance для GLM с распределением Пуассона

Метрика Deviance представляет собой отношение правдоподобия между двумя моделями: рассматриваемой моделью и "идеальной" моделью, в которая бы идеально предсказывала бы зависимую переменную.
$$Deviance = 2(\ell_{ideal} - \ell_{model})$$

В качестве такой "идеальной модели" может использоваться сама зависимая переменная. Тогда, логарифм правдоподобия "идеальной модели" для GLM с распределением Пуассона имеет вид:
$$\ell_{ideal} = \sum_{i=1}^{N}\left(y_i \ln(y_i) -y_i-\ln(y_i!)\right).$$

Из приведенного выше вывода правдоподобия для рассматриваемой модели, мы можем записать, обозначив $\hat{y}_i = e^{x{^T}\beta + {\beta_{0}}}$:
$$\ell_{model} = \sum_{i=1}^{N}\left(y_i \ln(\hat{y}_i) -\hat{y}_i-\ln(y_i!)\right).$$

Тогда получаем,
$$Deviance = 2\sum_{i=1}^{N}\left(y_i \ln(y_i) -y_i - y_i \ln(\hat{y}_i) +\hat{y}_i\right) = -2\sum_{i=1}^{N}\left(y_i \ln(y_i/\hat{y}_i) - (y_i -\hat{y}_i)\right).$$


#### GLM с гамма-распределением

GLM с гамма-распределением используется для моделирования положительной непрерывной зависимой переменной, когда ее условная дисперсия увеличивается вместе со средним значением, но коэффициент вариации зависимой переменной предполагается постоянным.

Обычно GLM с гамма-распределением используются с логарифмической или обратной функциями связи:
$$g(\eta) = \ln(\eta);\hspace{20pt}g(\eta) = \frac{1}{\eta}.$$

Модель оценивается методом максимального правдоподобия, функция логарифма правдоподобия (для обратной функции связи) с учетом штрафа регуляризации эластичной сети имеет вид:
$$\max_{\beta,\beta_0} - \frac{1}{N} \sum_{i=1}^{N} \frac{y_i}{x{^T_i}\beta + \beta_0} + \text{ln} \big( x{^T_i}\beta + \beta_0 \big ) - \lambda \Big( \alpha||\beta||_1 + \dfrac {1} {2}(1 - \alpha)||\beta||^2_2 \Big),$$

где 
* $\lambda$ – параметр, отвечающий за силу регуляризации. $\lambda\in\mathbb{R}^{+}$;
* $\alpha$ – параметр, отвечающий за распределение штрафов регуляризации между нормой 1 ($\ell_1$) и нормой 2 ($\ell_2$). $\alpha\in[0,1]$;
* $||\beta||{_1}$ – штраф регуляризации $\ell_1$ (LASSO). $||\beta||{_1} = \sum{^p_{k=1}} |\beta{_k}|$;
* $||\beta||{_2}$ – штраф регуляризации $\ell_2$ (Ridge). $||\beta||{_2} = \sum{^p_{k=1}} \beta{^2_k}$.

Соответствующая метрика _Deviance_ имеет вид:
$$D = 2 \sum_{i=1}^{N} - \text{ln} \bigg (\dfrac {y_i} {\hat {y}_i} \bigg) + \dfrac {(y_i - \hat{y}_i)} {\hat {y}_i}.$$




##### Вывод функции правдоподобия для GLM с Гамма-распределением и логарифмической функцией связи (без регуляризации)

**NB:** Ниже $\alpha$ и $\lambda$ не имеют отношения к вышеупомянутым одноименным параметрам регуляризации.

Напомним, что функция плотности вероятности для Гамма-распределения имеет вид:
$$f(x;\alpha,\lambda) = 
\begin{cases}
\frac{\alpha^\lambda}{\Gamma(\lambda)}x^{\lambda-1}e^{-\alpha x},&x \ge 0\\
0,& x <0
\end{cases},\hspace{20pt} \Gamma(x) = \int_0^{\infty}x^{\lambda-1}e^{-x}dx.
$$
Также, для Гамма-распределения справедливо, что:
$$ \mathbb{E}[x] = \frac{\lambda}{\alpha},\hspace{15pt} Var(x) = \frac{\lambda}{\alpha^2}.$$

Для оценивания GLM удобно параметризовать данное распределение иначе:
$$ \mathbb{E}[x] = \frac{\lambda}{\alpha} = \mu,\hspace{15pt} Var(x) = \frac{\lambda}{\alpha^2} = \frac{\mu^2}{\lambda}.$$
Тогда,
$$f(x;\mu,\lambda) = \frac{1}{\Gamma(\lambda)}\alpha^\lambda x^{\lambda-1}e^{-\alpha x} = \frac{1}{\Gamma(\lambda)}\left(\frac{\lambda}{\mu}\right)^\lambda x^{\lambda-1}e^{-\left(\frac{\lambda}{\mu}\right)x} = \frac{1}{x\cdot\Gamma(\lambda)}\left(\frac{\lambda x}{\mu}\right)^\lambda e^{-\frac{\lambda x}{\mu}},\hspace{15pt} x\ge 0.$$

Также напомним, что для GLM с функцией связи $\ln(x)$ предсказание (оценка математического ожидания) имеет вид $\hat {y} = e^{x{^T}\beta + {\beta_{0}}}$.


Тогда, для оценивания коэффициентов нашей модели необходимо максимизировать правдоподобие (совместную условную вероятность при имеющихся данных), что данные имеют Гамма-распределение, в предположении известного параметра $\lambda$:
$$p(y_1,\dots,y_n|x_1,\dots,x_n;\beta_0,\beta) = \prod_{i=1}^{N}\frac{1}{y_i\cdot\Gamma(\lambda)}\left(\frac{\lambda y_i}{e^{x_i{^T}\beta + {\beta_{0}}}}\right)^\lambda e^{-\frac{\lambda y_i}{e^{x_i{^T}\beta + {\beta_{0}}}}} = L(\beta_0,\beta|y_1,\dots,y_n,x_1,\dots,x_n).$$
Для упрощения задачи оптимизации перейдем к логарифму правдоподобия:
\begin{align}
\ell(\beta_0,\beta|y_1,\dots,y_n,x_1,\dots,x_n) &= \sum_{i=1}^{N}\left(-\ln(\Gamma(\lambda))-\ln(y_i)+ \lambda\ln\left(\frac{\lambda y_i}{e^{x_i{^T}\beta + {\beta_{0}}}}\right) -\frac{\lambda y_i}{e^{x_i{^T}\beta + {\beta_{0}}}}\right)\\ &= \sum_{i=1}^{N}\left(-\ln(\Gamma(\lambda))-\ln(y_i) + \lambda\ln\left(\lambda\right)+\lambda\ln\left(y_i\right)-\lambda\left({x_i{^T}\beta + {\beta_{0}}}\right) -\frac{\lambda y_i}{e^{x_i{^T}\beta + {\beta_{0}}}}\right)\\ &= \sum_{i=1}^{N}\left((\lambda-1)\ln\left(y_i\right)-\ln(\Gamma(\lambda)) + \lambda\ln\left(\lambda\right)-\lambda\left({x_i{^T}\beta + {\beta_{0}}}\right) -\frac{\lambda y_i}{e^{x_i{^T}\beta + {\beta_{0}}}}\right).
\end{align}
Тогда обозначим слагаемые, не зависящие от параметров модели ($\beta_0$, $\beta$), как некоторую константу $C$, получаем:
$$\ell(\beta_0,\beta|y_1,\dots,y_n,x_1,\dots,x_n) = \sum_{i=1}^{N}\left(-\lambda\left(\left({x_i{^T}\beta + {\beta_{0}}}\right) +\frac{y_i}{e^{x_i{^T}\beta + {\beta_{0}}}}\right) + C(y_i, \lambda)\right),$$
Таким образом, требуется максимизировать
$$\ell(\beta_0,\beta|y_1,\dots,y_n,x_1,\dots,x_n) = -\sum_{i=1}^{N}\left(\left({x_i{^T}\beta + {\beta_{0}}}\right) + \frac{y_i}{e^{x_i{^T}\beta + {\beta_{0}}}}\right),$$
Далее численно решается задача оптимизации для определения параметров модели:
$$\frac{\partial \ell(\beta_0,\beta|y_1,\dots,y_n,x_1,\dots,x_n)}{\partial \beta_0} = 0,\\\frac{\partial \ell(\beta_0,\beta|y_1,\dots,y_n,x_1,\dots,x_n)}{\partial \beta} = 0.$$
Обычно минимизируется отрицательное правдоподобие, которое является выпуклой функцией.

##### Вывод метрики Deviance для GLM с Гамма-распределением

Метрика Deviance представляет собой отношение правдоподобия между двумя моделями: рассматриваемой моделью и "идеальной" моделью, в которая бы идеально предсказывала бы зависимую переменную.
$$Deviance = 2(\ell_{ideal} - \ell_{model})$$

В качестве такой "идеальной модели" может использоваться сама зависимая переменная. Тогда, логарифм правдоподобия "идеальной модели" для GLM с распределением Пуассона имеет вид:
$$\ell_{ideal} = -\sum_{i=1}^{N}\left(1 + \ln(y_i)\right).$$

Из приведенного выше вывода правдоподобия для рассматриваемой модели, мы можем записать, обозначив $\hat{y}_i = e^{x{^T}\beta + {\beta_{0}}}$:
$$\ell_{model} = -\sum_{i=1}^{N}\left(\frac{y_i}{\hat{y}_i} + \ln(\hat{y}_i)\right).$$

Тогда получаем,
$$Deviance = 2\sum_{i=1}^{N}\left(- 1 - \ln(y_i) + \frac{y_i}{\hat{y}_i} + \ln(\hat{y}_i)\right) = 2\sum_{i=1}^{N}\left(-\ln\left(\frac{y_i}{\hat{y}_i}\right)+\frac{y_i-\hat{y}_i}{\hat{y}_i} \right).$$

#### Дополнительная литература по GLM

*   [P. McCullagh, John A. Nelder _"Generalized Linear Models"_](http://www.utstat.toronto.edu/~brunner/oldclass/2201s11/readings/glmbook.pdf)
*   [D. Anderson, et al. _"A practitioner’s guide to generalized linear models"_](https://www.casact.org/pubs/dpp/dpp04/04dpp1.pdf)
*   [E. Ohlsson, B. Johansson _"Non-life insurance pricing with generalized linear models"_](https://www.springer.com/gp/book/9783642107900)
*   [P. De Jong, G. Heller _"Generalized linear models for insurance data"_](https://feb.kuleuven.be/public/u0017833/boek.pdf)




### Установка H2O на Google Colaboratory и инициализация

In [25]:
# !apt-get install default-jre

In [26]:
# !java -version

In [27]:
# !pip install h2o

In [28]:
import h2o
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 12.0.1+12, mixed mode, sharing)
  Starting server from C:\Users\TOSHIBA\Anaconda3\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\TOSHIBA\AppData\Local\Temp\tmp11a9t11i
  JVM stdout: C:\Users\TOSHIBA\AppData\Local\Temp\tmp11a9t11i\h2o_TOSHIBA_started_from_python.out
  JVM stderr: C:\Users\TOSHIBA\AppData\Local\Temp\tmp11a9t11i\h2o_TOSHIBA_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,07 secs
H2O cluster timezone:,+03:00
H2O data parsing timezone:,UTC
H2O cluster version:,3.28.0.1
H2O cluster version age:,6 days
H2O cluster name:,H2O_from_python_TOSHIBA_ls3kba
H2O cluster total nodes:,1
H2O cluster free memory:,1000 Mb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"accepting new members, healthy"


### Построение GLM для частоты страховых случаев

In [29]:
# Преобразование в H2O-Frame

h2o_train_c = h2o.H2OFrame(pd.concat([x_train_c, y_train_c], axis=1))
h2o_valid_c = h2o.H2OFrame(pd.concat([x_valid_c, y_valid_c], axis=1))
h2o_test_c = h2o.H2OFrame(pd.concat([x_test_c, y_test_c], axis=1))

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [30]:
print(h2o_train_c.names)

['Exposure', 'LicAge', 'Gender', 'MariStat', 'DrivAge', 'HasKmLimit', 'BonusMalus', 'OutUseNb', 'RiskArea', 'VehUsg_Private', 'VehUsg_Private+trip to office', 'VehUsg_Professional', 'VehUsg_Professional run', 'CSP1', 'CSP2', 'CSP3', 'CSP4', 'CSP5', 'CSP6', 'CSP7', 'DrivAgeSq', 'ClaimsCount']


In [31]:
# Инициализируем и обучим GLM модель c кросс-валидацией

glm_poisson = H2OGeneralizedLinearEstimator(family = "poisson", link = "Log", nfolds=5)
glm_poisson.train(y="ClaimsCount", x = h2o_train_c.names[1:-1], training_frame = h2o_train_c, validation_frame = h2o_valid_c, weights_column = "Exposure")

glm Model Build progress: |███████████████████████████████████████████████| 100%


In [32]:
# Параметры модели: распределение, функция связи, гиперпараметры регуляризации, количество использованных объясняющих переменных

glm_poisson.summary()


GLM Model: summary


,,family,link,regularization,number_of_predictors_total,number_of_active_predictors,number_of_iterations,training_frame
0,,poisson,log,"Elastic Net (alpha = 0.5, lambda = 1.308E-4 )",20,19,3,Key_Frame__upload_8e40251a520d85e5f2c5d0334ca84f8c.hex


In [33]:
# Метрики качества модели - по всем данным и на кросс-валидации

glm_poisson.cross_validation_metrics_summary().as_data_frame()

,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,mae,0.5026557,0.0025469651,0.50318086,0.49816754,0.5037342,0.503777,0.50441873
1,mean_residual_deviance,1.2425531,0.015562582,1.2460407,1.2152565,1.2479541,1.2492174,1.2542971
2,mse,0.7540859,0.019869654,0.74926496,0.72166336,0.767731,0.7614064,0.77036387
3,null_deviance,9371.563,102.61999,9459.307,9207.098,9349.936,9388.558,9452.922
4,r2,0.0125999525,0.0012280245,0.011119368,0.011963396,0.014427061,0.012909975,0.012579962
5,residual_deviance,9133.998,105.27886,9242.046,8992.435,9064.389,9150.51,9220.611
6,rmse,0.8683202,0.011507436,0.86560094,0.8495077,0.8762026,0.8725861,0.8777037
7,rmsle,0.4100735,0.0020037,0.41094837,0.40650553,0.41067347,0.4110395,0.41120052


In [34]:
# Таблица коэффициентов модели (в зависимости от модели могут выводиться также стандартная ошибка, z-score и p-value)

glm_poisson._model_json['output']['coefficients_table'].as_data_frame()

,names,coefficients,standardized_coefficients
0,Intercept,-2.232571,-1.289184
1,LicAge,0.000046,0.007411
2,Gender,-0.020411,-0.009888
3,MariStat,-0.127848,-0.045723
4,DrivAge,0.006846,0.102613
5,HasKmLimit,-0.451913,-0.139946
6,BonusMalus,0.012746,0.192535
7,OutUseNb,0.095582,0.064078
8,RiskArea,0.013265,0.029377
9,VehUsg_Private,-0.232444,-0.110219


In [35]:
# Таблица нормированных коэффициентов по всем данным и на кросс-валидации

pmodels = {}
pmodels['overall'] = glm_poisson.coef_norm()
for x in range(len(glm_poisson.cross_validation_models())):
    pmodels[x] = glm_poisson.cross_validation_models()[x].coef_norm()
pd.DataFrame.from_dict(pmodels).round(5)

,overall,0,1,2,3,4
BonusMalus,0.19253,0.19272,0.19500,0.19388,0.18716,0.19145
CSP1,-0.02652,-0.03299,-0.03104,-0.02481,-0.02339,-0.03052
CSP2,-0.05998,-0.05937,-0.06678,-0.05626,-0.06378,-0.06543
CSP3,0.03577,0.03130,0.03131,0.04234,0.03150,0.03561
CSP4,0.01363,0.00252,0.02246,0.01355,0.01224,0.00077
CSP5,-0.00567,-0.00535,-0.01494,0.00000,-0.02058,-0.01709
CSP6,0.03688,0.02172,0.03805,0.05037,0.01569,0.02576
CSP7,-0.01064,-0.00859,-0.03899,-0.00804,-0.01102,-0.00543
DrivAge,0.10261,0.12609,0.04942,0.16978,0.01630,0.06928
DrivAgeSq,-0.10989,-0.11166,-0.11085,-0.16456,-0.02017,-0.06821


In [36]:
# Построение прогнозных значений для обучающей, валидационной и тестовой выборок

c_train_pred = glm_poisson.predict(h2o_train_c).as_data_frame()
c_valid_pred = glm_poisson.predict(h2o_valid_c).as_data_frame()
c_test_pred = glm_poisson.predict(h2o_test_c).as_data_frame()

glm prediction progress: |████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%


### Построение GLM для среднего убытка

In [37]:
# Преобразование в H2O-Frame

h2o_train_ac = h2o.H2OFrame(pd.concat([x_train_ac, y_train_ac], axis=1))
h2o_valid_ac = h2o.H2OFrame(pd.concat([x_valid_ac, y_valid_ac], axis=1))
h2o_test_ac = h2o.H2OFrame(pd.concat([x_test_ac, y_test_ac], axis=1))

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [38]:
print(h2o_train_ac.names)

['Exposure', 'LicAge', 'Gender', 'MariStat', 'DrivAge', 'HasKmLimit', 'BonusMalus', 'OutUseNb', 'RiskArea', 'VehUsg_Private', 'VehUsg_Private+trip to office', 'VehUsg_Professional', 'VehUsg_Professional run', 'CSP1', 'CSP2', 'CSP3', 'CSP4', 'CSP5', 'CSP6', 'CSP7', 'DrivAgeSq', 'AvgClaim']


In [39]:
# Инициализируем и обучим GLM модель c кросс-валидацией

glm_gamma = H2OGeneralizedLinearEstimator(family = "gamma", link = "Log", nfolds=5)
glm_gamma.train(y="AvgClaim", x = h2o_train_ac.names[1:-1], training_frame = h2o_train_ac, validation_frame = h2o_valid_ac, weights_column = "Exposure")

glm Model Build progress: |███████████████████████████████████████████████| 100%


In [40]:
# Параметры модели: распределение, функция связи, гиперпараметры регуляризации, количество использованных объясняющих переменных

glm_gamma.summary()


GLM Model: summary


,,family,link,regularization,number_of_predictors_total,number_of_active_predictors,number_of_iterations,training_frame
0,,gamma,log,"Elastic Net (alpha = 0.5, lambda = 1.782E-4 )",20,19,3,Key_Frame__upload_bf8c117bbddb344733030204ff2d4a0f.hex


In [41]:
# Метрики качества модели - по всем данным и на кросс-валидации

glm_gamma.cross_validation_metrics_summary().as_data_frame()

,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,mae,1045.9604,24.55808,1038.861,1023.04346,1024.5453,1066.4042,1076.9484
1,mean_residual_deviance,1.9645181,0.06107039,1.9602493,1.947963,1.8748348,2.0352159,2.0043275
2,mse,7260119.5,4980672.5,4618186.5,1.5757304E7,3589593.8,4720834.5,7614680.5
3,null_deviance,1853.8655,74.27506,1785.3132,1847.2856,1778.073,1919.5597,1939.0957
4,r2,-0.0018745367,0.007098992,-3.710864E-4,0.0016175916,-0.011903862,-0.0054351124,0.0067197853
5,residual_deviance,1857.5271,71.93383,1780.273,1825.8102,1821.7488,1961.2664,1898.5371
6,rmse,2589.077,834.26605,2148.9966,3969.547,1894.6223,2172.748,2759.471
7,rmsle,1.7124239,0.034563787,1.7050155,1.7342614,1.6550205,1.7301483,1.7376738


In [42]:
# Таблица коэффициентов модели (в зависимости от модели могут выводиться также стандартная ошибка, z-score и p-value)

glm_gamma._model_json['output']['coefficients_table'].as_data_frame()

,names,coefficients,standardized_coefficients
0,Intercept,7.622904,6.920750
1,LicAge,-0.000346,-0.054870
2,Gender,-0.053660,-0.025964
3,MariStat,0.150567,0.055406
4,DrivAge,-0.035689,-0.524162
5,HasKmLimit,-0.055473,-0.013748
6,BonusMalus,-0.002959,-0.048894
7,OutUseNb,0.014387,0.010521
8,RiskArea,0.042049,0.094112
9,VehUsg_Private,0.040138,0.018185


In [43]:
# Таблица нормированных коэффициентов по всем данным и на кросс-валидации

pmodels = {}
pmodels['overall'] = glm_gamma.coef_norm()
for x in range(len(glm_gamma.cross_validation_models())):
    pmodels[x] = glm_gamma.cross_validation_models()[x].coef_norm()
pd.DataFrame.from_dict(pmodels).round(5)

,overall,0,1,2,3,4
BonusMalus,-0.04889,-0.04691,-0.02917,-0.04421,-0.06461,-0.05820
CSP1,0.01782,-0.00261,0.03583,0.00607,0.02747,0.01482
CSP2,-0.02052,-0.02105,-0.00715,-0.00051,-0.07300,-0.01895
CSP3,-0.01708,-0.00428,-0.01973,-0.02315,-0.02119,-0.02188
CSP4,-0.01175,-0.01700,-0.00090,-0.00557,-0.04554,0.00000
CSP5,0.02452,0.00935,0.04751,0.01776,0.00840,0.01687
CSP6,-0.00209,0.00194,0.00000,0.00000,0.00000,-0.03310
CSP7,-0.01509,-0.01776,-0.01527,-0.01531,-0.01697,0.00000
DrivAge,-0.52416,-0.54430,-0.45112,-0.50616,-0.60586,-0.54374
DrivAgeSq,0.54753,0.55399,0.47213,0.56383,0.59989,0.57963


In [44]:
# Построение прогнозных значений для обучающей, валидационной и тестовой выборок

ac_train_pred = glm_gamma.predict(h2o_train_ac).as_data_frame()
ac_valid_pred = glm_gamma.predict(h2o_valid_ac).as_data_frame()
ac_test_pred = glm_gamma.predict(h2o_test_ac).as_data_frame()

glm prediction progress: |████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%


### Использование GLM моделей

In [45]:
h2o_df = h2o.H2OFrame(df)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [46]:
df['CountPredicted'] = glm_poisson.predict(h2o_df).as_data_frame()
df['AvgClaimPredicted'] = glm_gamma.predict(h2o_df).as_data_frame()

glm prediction progress: |████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%


In [47]:
df['BurningCost'] = df.CountPredicted * df.AvgClaimPredicted
df[['CountPredicted', 'AvgClaimPredicted', 'BurningCost']].head()

,CountPredicted,AvgClaimPredicted,BurningCost
0,0.375992,988.222409,371.563352
1,0.376009,987.880713,371.452138
2,0.322919,1039.015533,335.517481
3,0.261780,926.138283,242.444676
4,0.230704,923.064855,212.954902


## * Домашнее задание: GLM для прогнозирования наступления страхового случая

In [48]:
# Разбиение датасета на train/val/test

x_train_ind, x_test_ind, y_train_ind, y_test_ind = train_test_split(df.drop(['ClaimInd', 'ClaimAmount', 'ClaimsCount', 'CountPredicted', 'AvgClaimPredicted', 'BurningCost'], axis=1), df.ClaimInd, test_size=0.15, random_state=1)
x_train_ind, x_valid_ind, y_train_ind, y_valid_ind = train_test_split(x_train_ind, y_train_ind, test_size=0.15, random_state=1)

In [49]:
# Преобразование в H2O-Frame

h2o_train_ind = h2o.H2OFrame(pd.concat([x_train_ind, y_train_ind], axis=1))
h2o_test_ind = h2o.H2OFrame(pd.concat([x_test_ind, y_test_ind], axis=1))
h2o_valid_ind = h2o.H2OFrame(pd.concat([x_valid_ind, y_valid_ind], axis=1))

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [50]:
# Преобразуем целевую переменную ClaimInd в категориальную при помощи метода asfactor во всех наборах данных

h2o_test_ind['ClaimInd'] = h2o_test_ind['ClaimInd'].asfactor()
h2o_train_ind['ClaimInd'] = h2o_train_ind['ClaimInd'].asfactor()
h2o_valid_ind['ClaimInd'] = h2o_valid_ind['ClaimInd'].asfactor()

In [51]:
print(df['ClaimInd'].unique())

[0 1]


Классов всего два (1 и 0: наступил или не наступил страховой случай), поэтому выбраны логистическая регрессия и биномиальное распределение. Согласно документации http://docs.h2o.ai/h2o/latest-stable/h2o-docs/booklets/GLMBooklet.pdf настройка прозводится так:

In [52]:
# Инициализируем и обучим GLM модель c кросс-валидацией

glm_binom = H2OGeneralizedLinearEstimator(family = "binomial", nfolds=5)
glm_binom.train(y="ClaimInd", x=h2o_train_ind.names[1:-1],
                training_frame=h2o_train_ind,
                validation_frame=h2o_valid_ind,
                weights_column = "Exposure")

glm Model Build progress: |███████████████████████████████████████████████| 100%


In [53]:
# Параметры модели: распределение, функция связи, гиперпараметры регуляризации, количество использованных объясняющих переменных

glm_binom.summary()


GLM Model: summary


,,family,link,regularization,number_of_predictors_total,number_of_active_predictors,number_of_iterations,training_frame
0,,binomial,logit,"Elastic Net (alpha = 0.5, lambda = 3.226E-5 )",20,20,3,py_1_sid_870d


In [54]:
# Метрики качества модели - по всем данным и на кросс-валидации

glm_binom.cross_validation_metrics_summary().as_data_frame()

,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,accuracy,0.46671304,0.09610284,0.5338987,0.48677325,0.31148627,0.44769102,0.55371594
1,auc,0.5704972,0.0128757665,0.5893784,0.5555718,0.56919867,0.5628974,0.5754399
2,aucpr,0.15611616,0.010768016,0.1729398,0.1434483,0.15572163,0.15168,0.15679105
3,err,0.5332869,0.09610284,0.4661013,0.51322675,0.68851376,0.552309,0.44628403
4,err_count,3922.1694,720.9672,3408.003,3789.36,5088.239,4051.674,3273.571
5,f0point5,0.17619686,0.012831638,0.19675523,0.16639318,0.16867301,0.16833572,0.18082713
6,f1,0.24234556,0.013515995,0.26437718,0.22901195,0.24130747,0.23420651,0.24282469
7,f2,0.389623,0.02384098,0.40282097,0.36720064,0.42380956,0.38476887,0.36951482
8,lift_top_group,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,logloss,0.37895122,0.007441503,0.3916695,0.37340719,0.37934417,0.37483028,0.37550503


In [55]:
# Таблица коэффициентов модели (в зависимости от модели могут выводиться также стандартная ошибка, z-score и p-value)

glm_binom._model_json['output']['coefficients_table'].as_data_frame()

,names,coefficients,standardized_coefficients
0,Intercept,-2.281283,-1.945709
1,LicAge,-0.000124,-0.019844
2,Gender,-0.006755,-0.003273
3,MariStat,-0.059439,-0.021257
4,DrivAge,-0.004067,-0.060961
5,HasKmLimit,-0.412808,-0.127837
6,BonusMalus,0.006568,0.099217
7,OutUseNb,0.087129,0.058411
8,RiskArea,0.008654,0.019165
9,VehUsg_Private,-0.156076,-0.074007


In [56]:
# Таблица нормированных коэффициентов по всем данным и на кросс-валидации

pmodels = {}
pmodels['overall'] = glm_binom.coef_norm()
for x in range(len(glm_binom.cross_validation_models())):
    pmodels[x] = glm_binom.cross_validation_models()[x].coef_norm()
pd.DataFrame.from_dict(pmodels).round(5)

,overall,0,1,2,3,4
BonusMalus,0.09922,0.08944,0.10196,0.10819,0.09686,0.10167
CSP1,-0.00920,-0.02314,-0.03034,-0.03560,-0.02065,-0.01603
CSP2,-0.02396,-0.03565,-0.04162,-0.03552,-0.05767,-0.03694
CSP3,0.04142,0.01769,0.03142,0.04186,0.03064,0.03052
CSP4,0.04315,0.01828,0.00751,0.02810,0.01557,0.01572
CSP5,0.05673,0.00679,0.01152,0.01608,0.00000,0.00000
CSP6,0.08372,0.02347,0.04669,0.04609,0.03786,0.05076
CSP7,-0.00626,-0.00515,-0.00200,-0.00408,-0.00414,-0.04376
DrivAge,-0.06096,-0.07694,-0.03331,-0.07133,-0.02718,-0.02869
DrivAgeSq,0.02358,0.06069,-0.03067,0.03773,0.00803,-0.01966


In [57]:
# Построение прогнозных значений для обучающей, валидационной и тестовой выборок

ind_train_pred = glm_binom.predict(h2o_train_ind).as_data_frame()
ind_test_pred = glm_binom.predict(h2o_test_ind).as_data_frame()
ind_valid_pred = glm_binom.predict(h2o_valid_ind).as_data_frame()

glm prediction progress: |████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%


In [58]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

In [59]:
# Выведем импортированные выше метрики классификации для обучающей, валидационной и тестовой выборок

print(f'TRAIN\n accuracy: {accuracy_score(y_train_ind, ind_train_pred.predict):.6f}' + \
      f'\n       f1: {f1_score(y_train_ind, ind_train_pred.predict):.6f}\n' + \
      f'\n confusion_matrix:\n{confusion_matrix(y_train_ind, ind_train_pred.predict)}\n\n')

print(f'TEST\n accuracy: {accuracy_score(y_test_ind, ind_test_pred.predict):.6f}' + \
      f'\n       f1: {f1_score(y_test_ind, ind_test_pred.predict):.6f}\n' + \
      f'\n confusion_matrix:\n{confusion_matrix(y_test_ind, ind_test_pred.predict)}\n\n')

print(f'VALID\n accuracy: {accuracy_score(y_valid_ind, ind_valid_pred.predict):.6f}' + \
      f'\n       f1: {f1_score(y_valid_ind, ind_valid_pred.predict):.6f}\n' + \
      f'\n confusion_matrix:\n{confusion_matrix(y_valid_ind, ind_valid_pred.predict)}')

TRAIN
 accuracy: 0.316798
       f1: 0.182838

 confusion_matrix:
[[19998 55351]
 [ 1490  6359]]


TEST
 accuracy: 0.313535
       f1: 0.180511

 confusion_matrix:
[[ 4110 11550]
 [  308  1306]]


VALID
 accuracy: 0.318327
       f1: 0.183805

 confusion_matrix:
[[3547 9730]
 [ 279 1127]]


Какие проблемы вы здесь видите? Как можно улучшить данный результат?

In [60]:
# доли классов
df['ClaimInd'].value_counts(normalize=True)

0    0.905614
1    0.094386
Name: ClaimInd, dtype: float64

В данных присутствует дисбаланс классов - очень мало записей, когда страховой случай наступал. Этим объясняются низкие показатели доли правильных ответов (в три раза ниже доли самого большого класса) и $F1$-меры, а также большие количества False Negative примеров в матрице ошибок (1 как 0). Для улучшения результата перед обучением модели необходимо предпринять меры по ликвидации дисбаланса (стратегии oversampling, undersampling или reweighting).